In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import math

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seed for reproducibility
torch.manual_seed(64)

# Path for saving models and results
save_path = 'Training_results/Harvey_5foldcv_BERT_Multihead_Deeper_512/'
os.makedirs(save_path, exist_ok=True)

# Load BERT tokenizer and model
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

def haversine(lat1, lon1, lat2, lon2):
    R = 3959  
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2

# Get the target list if not provided
def get_target_list(target_list=[]):
    if not target_list:
        target_list = [
            'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage', 
            'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible', 
            'replacementAssistanceEligible', 'personalPropertyEligible'
        ]
    return target_list

# Define dataset class to handle tokenization and data loading
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenizing the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.float32)
        }

class BERTMultiDeepHeadClassifier(nn.Module):
    def __init__(self, num_targets=11):
        super(BERTMultiDeepHeadClassifier, self).__init__()
        self.bert = model_bert
        self.drop = nn.Dropout(0.3)
        
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(128, 1),
                nn.Sigmoid()
            ) for _ in range(num_targets)
        ])

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = self.drop(outputs.pooler_output)
        return torch.cat([head(pooled_output) for head in self.heads], dim=1)

def kfoldcv(grouped_tweets, grouped_targets, target_list=[], max_len=256, batch_size=32, num_epochs=10, patience=15):
    saved_models = []
    target_list = get_target_list(target_list)
    
    tweet_dict = {int(name): group['text'] for name, group in grouped_tweets}
    target_dict = {int(name): group[target_list] for name, group in grouped_targets}
    
    intersecting_zips = list(set(target_dict.keys()) & set(tweet_dict.keys()))
    paired_data = {
        name: [target_dict[name].sum().apply(lambda x: 1 if x > 0 else 0), tweet_dict[name]] for name in intersecting_zips
    }
    
    texts = ['\n'.join(v[1].to_list()) for v in paired_data.values()]
    labels_ = [v[0] for v in paired_data.values()]
    
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    best_f1 = -float('inf')
    
    for fold, (train_idx, test_idx) in enumerate(kfold.split(texts, labels_)):
        print(f"Training fold {fold + 1}...")

        X_train, X_test = [texts[i] for i in train_idx], [texts[i] for i in test_idx]
        y_train, y_test = [labels_[i] for i in train_idx], [labels_[i] for i in test_idx]
        
        train_dataset = CustomDataset(X_train, y_train, tokenizer, max_len)
        test_dataset = CustomDataset(X_test, y_test, tokenizer, max_len)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        model = BERTMultiDeepHeadClassifier(num_targets=len(target_list)).to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-5)
        criterion = nn.BCELoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)
        
        early_stop_count = 0
        best_model_wts = copy.deepcopy(model.state_dict())
        best_epoch_f1 = 0
        
        for epoch in range(num_epochs):
            model.train()
            train_loss = 0
            train_preds, train_labels = [], []
            
            with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
                for batch in tepoch:
                    optimizer.zero_grad()
                    
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)
                    
                    outputs = model(input_ids, attention_mask)
                    loss = sum([criterion(outputs[:, i], labels[:, i]) for i in range(len(target_list))])
                    
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    
                    tepoch.set_postfix(loss=loss.item())
                    
                    train_preds.append(outputs.detach().cpu().numpy())
                    train_labels.append(labels.cpu().numpy())
            
            train_preds = np.vstack(train_preds)
            train_labels = np.vstack(train_labels)
            
            model.eval()
            val_loss = 0
            val_preds, val_labels = [], []
            
            with torch.no_grad():
                for batch in tqdm(test_loader, desc="Validating", unit="batch"):
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)
                    
                    outputs = model(input_ids, attention_mask)
                    loss = sum([criterion(outputs[:, i], labels[:, i]) for i in range(len(target_list))])
                    
                    val_loss += loss.item()
                    val_preds.append(outputs.cpu().numpy())
                    val_labels.append(labels.cpu().numpy())
            
            val_preds = np.vstack(val_preds)
            val_labels = np.vstack(val_labels)
            
            f1 = np.mean([f1_score(val_labels[:, i], val_preds[:, i] > 0.5) for i in range(len(target_list))])
            accuracy = np.mean([accuracy_score(val_labels[:, i], val_preds[:, i] > 0.5) for i in range(len(target_list))])
            precision = np.mean([precision_score(val_labels[:, i], val_preds[:, i] > 0.5) for i in range(len(target_list))])
            recall = np.mean([recall_score(val_labels[:, i], val_preds[:, i] > 0.5) for i in range(len(target_list))])
            
            print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, F1: {f1:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
            
            scheduler.step(f1)
            
            if f1 > best_epoch_f1:
                best_epoch_f1 = f1
                best_model_wts = copy.deepcopy(model.state_dict())
                early_stop_count = 0
            else:
                early_stop_count += 1
                if early_stop_count >= patience:
                    print("Early stopping triggered.")
                    break
        
        model.load_state_dict(best_model_wts)
        torch.save(model.state_dict(), os.path.join(save_path, f"bert_model_fold_{fold+1}.pth"))
        saved_models.append(model)

In [2]:
size_threshold = 80

tweets_harvey2 = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/tweets/harvey_corrected.csv')
tweets_harvey2.rename(columns={'zipcode': 'zip_code'}, inplace=True)

bboxes_useful = tweets_harvey2.place_bbox.apply(lambda x: [[float(i.strip('()[]')) for i in x.split(', ')][i] for i in [1,0,3,2]])
bbu_areas = bboxes_useful.apply(lambda x: get_box_area(*x))
tweets_harvey = tweets_harvey2.loc[((tweets_harvey2.geo.apply(lambda x: 'Point' in str(x))) | (bbu_areas < size_threshold)), :]
tweets_harvey.loc[:, 'zip_code'] = tweets_harvey['zip_code'].apply(int)
tweet_grouped_harvey = tweets_harvey.groupby('zip_code')

targets_harvey = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/targets/disaster_4332.csv')
target_grouped_harvey = targets_harvey.groupby('damagedZipCode')

# After running the kfoldcv function:
kfoldcv(tweet_grouped_harvey, target_grouped_harvey, max_len = 512, batch_size=24, num_epochs = 100)

C:\Users\udays\AppData\Local\Temp\ipykernel_15096\1286408893.py:12: DtypeWarning: Columns (9,10,13,28,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  targets_harvey = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/targets/disaster_4332.csv')


Training fold 1...


C:\Users\udays\miniconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\udays\AppData\Local\Temp\ipykernel_15096\3760703809.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Validating: 100%|█████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.63batch/s]
C:\Users\udays\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1: Train Loss: 105.7690, Val Loss: 29.5674, F1: 0.6401, Accuracy: 0.6741, Precision: 0.5284, Recall: 0.8164


C:\Users\udays\AppData\Local\Temp\ipykernel_15096\3760703809.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Validating: 100%|█████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.73batch/s]
C:\Users\udays\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\udays\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter 

...

Epoch 15: Train Loss: 50.4374, Val Loss: 15.1728, F1: 0.7785, Accuracy: 0.8967, Precision: 0.7684, Recall: 0.7922


C:\Users\udays\AppData\Local\Temp\ipykernel_15096\3760703809.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Validating: 100%|█████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.38batch/s]
C:\Users\udays\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\udays\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter 

Epoch 16: Train Loss: 49.8949, Val Loss: 15.1005, F1: 0.7785, Accuracy: 0.8967, Precision: 0.7684, Recall: 0.7922


C:\Users\udays\AppData\Local\Temp\ipykernel_15096\3760703809.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Validating: 100%|█████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.38batch/s]
C:\Users\udays\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\udays\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter 

Epoch 17: Train Loss: 49.7216, Val Loss: 15.0822, F1: 0.7785, Accuracy: 0.8967, Precision: 0.7684, Recall: 0.7922
Early stopping triggered.


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import math

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seed for reproducibility
torch.manual_seed(64)

# Path for saving models and results
save_path = 'Training_results/Harvey_5foldcv_BERT_Multihead/'
os.makedirs(save_path, exist_ok=True)

# Load BERT tokenizer and model from HuggingFace's transformers library
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

def haversine(lat1, lon1, lat2, lon2):
    R = 3959  
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2

# Get the target list if not provided
def get_target_list(target_list=[]):
    if not target_list:
        target_list = [
            'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage', 
            'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible', 
            'replacementAssistanceEligible', 'personalPropertyEligible'
        ]
    return target_list

# Define dataset class to handle tokenization and data loading
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenizing the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.float32)
        }

class BERTDeepMultiHeadClassifier(nn.Module):
    def __init__(self, num_targets=11, hidden_dim=256):
        super(BERTDeepMultiHeadClassifier, self).__init__()
        self.bert = model_bert
        self.drop = nn.Dropout(0.3)
        
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_targets)
        ])
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.drop(pooled_output)
        return torch.cat([torch.sigmoid(head(x)) for head in self.heads], dim=1)

def kfoldcv(grouped_tweets, grouped_targets, target_list=[], max_len=256, batch_size=32, num_epochs=3):
    saved_models = []
    target_list = get_target_list(target_list)
    
    tweet_dict = {int(name): group['text'] for name, group in grouped_tweets}
    target_dict = {int(name): group[target_list] for name, group in grouped_targets}
    
    intersecting_zips = list(set(target_dict.keys()) & set(tweet_dict.keys()))
    paired_data = {
        name: [target_dict[name].sum().apply(lambda x: 1 if x > 0 else 0), tweet_dict[name]] for name in intersecting_zips
    }
    
    texts = ['\n'.join(v[1].to_list()) for v in paired_data.values()]
    labels_ = [v[0] for v in paired_data.values()]
    
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_results = []

    # print(len(texts), len(labels))
    
    for fold, (train_idx, test_idx) in enumerate(kfold.split(texts, labels_)):
        # print(f"Train indices: {train_idx}")
        # print(f"Test indices: {test_idx}")
        print(f"Training fold {fold + 1}...")

        X_train, X_test = [texts[i] for i in train_idx], [texts[i] for i in test_idx]
        y_train, y_test = [labels_[i] for i in train_idx], [labels_[i] for i in test_idx]
        
        train_dataset = CustomDataset(X_train, y_train, tokenizer, max_len)
        test_dataset = CustomDataset(X_test, y_test, tokenizer, max_len)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        model = BERTDeepMultiHeadClassifier(num_targets=len(target_list)).to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-5)
        criterion = [nn.BCELoss() for _ in range(len(target_list))]
        
        best_f1 = 0.0  # Track the best validation F1 score
        
        for epoch in range(num_epochs):
            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} Training")
            for batch in progress_bar:
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids, attention_mask)
                loss = sum(c(outputs[:, i], labels[:, i]) for i, c in enumerate(criterion))
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix(loss=loss.item())
            
            model.eval()
            val_loss = 0.0
            all_preds, all_labels = [], []
            progress_bar = tqdm(test_loader, desc=f"Epoch {epoch+1} Validation")
            with torch.no_grad():
                for batch in progress_bar:
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)
                    
                    outputs = model(input_ids, attention_mask)
                    loss = sum(c(outputs[:, i], labels[:, i]) for i, c in enumerate(criterion))
                    val_loss += loss.item()
                    
                    all_preds.append(outputs.cpu().numpy())
                    all_labels.append(labels.cpu().numpy())
                    progress_bar.set_postfix(loss=loss.item())
            
            all_preds = np.vstack(all_preds)
            all_labels = np.vstack(all_labels)
            f1 = [f1_score(all_labels[:, i], all_preds[:, i] > 0.5) for i in range(len(target_list))]
            accuracy = [accuracy_score(all_labels[:, i], all_preds[:, i] > 0.5) for i in range(len(target_list))]
            precision = [precision_score(all_labels[:, i], all_preds[:, i] > 0.5, zero_division=0) for i in range(len(target_list))]
            recall = [recall_score(all_labels[:, i], all_preds[:, i] > 0.5) for i in range(len(target_list))]
            mean_f1 = np.mean(f1)
            mean_accuracy = np.mean(accuracy)
            mean_precision = np.mean(precision)
            mean_recall = np.mean(recall)
            
            print(f"Epoch {epoch+1}: Train Loss={train_loss/len(train_loader):.4f}, Val Loss={val_loss/len(test_loader):.4f}")
            print(f"Mean F1 Score: {mean_f1:.4f}, Mean Accuracy: {mean_accuracy:.4f}, Mean Precision: {mean_precision:.4f}, Mean Recall: {mean_recall:.4f}")
            
            # Save the best model based on validation F1 score
            if mean_f1 > best_f1:
                best_f1 = mean_f1
                best_model_path = os.path.join(save_path, f"best_bert_model_fold_{fold+1}.pth")
                torch.save(model.state_dict(), best_model_path)
                print(f"New best model saved for fold {fold+1} with F1={best_f1:.4f}")

        saved_models.append(best_model_path)
    
    return fold_results, saved_models

In [2]:
size_threshold = 80

tweets_harvey2 = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/tweets/harvey_corrected.csv')
tweets_harvey2.rename(columns={'zipcode': 'zip_code'}, inplace=True)

bboxes_useful = tweets_harvey2.place_bbox.apply(lambda x: [[float(i.strip('()[]')) for i in x.split(', ')][i] for i in [1,0,3,2]])
bbu_areas = bboxes_useful.apply(lambda x: get_box_area(*x))
tweets_harvey = tweets_harvey2.loc[((tweets_harvey2.geo.apply(lambda x: 'Point' in str(x))) | (bbu_areas < size_threshold)), :]
tweets_harvey.loc[:, 'zip_code'] = tweets_harvey['zip_code'].apply(int)
tweet_grouped_harvey = tweets_harvey.groupby('zip_code')

targets_harvey = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/targets/disaster_4332.csv')
target_grouped_harvey = targets_harvey.groupby('damagedZipCode')

# X_dict_harvey = tweet_data_to_X_dict(grouped_tweets_to_dict(tweet_grouped_harvey), hub_layer)

# After running the kfoldcv function:
fold_results, saved_models = kfoldcv(tweet_grouped_harvey, target_grouped_harvey, batch_size=32, num_epochs = 100)

C:\Users\udays\AppData\Local\Temp\ipykernel_22228\2613428792.py:12: DtypeWarning: Columns (9,10,13,28,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  targets_harvey = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/targets/disaster_4332.csv')


Training fold 1...


C:\Users\udays\AppData\Local\Temp\ipykernel_22228\3925381797.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 1 Validation: 100%|█████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.94it/s, loss=6.97]


Epoch 1: Train Loss=7.5111, Val Loss=7.1423
Mean F1 Score: 0.5687, Mean Accuracy: 0.6519, Mean Precision: 0.4690, Mean Recall: 0.7273
New best model saved for fold 1 with F1=0.5687


C:\Users\udays\AppData\Local\Temp\ipykernel_22228\3925381797.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 2 Validation: 100%|█████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.02it/s, loss=6.53]


...

Epoch 98: Train Loss=0.2457, Val Loss=0.3023
Mean F1 Score: 0.9915, Mean Accuracy: 0.9888, Mean Precision: 0.9885, Mean Recall: 0.9947


C:\Users\udays\AppData\Local\Temp\ipykernel_22228\3925381797.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 99 Validation: 100%|███████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.39it/s, loss=0.073]


Epoch 99: Train Loss=0.2420, Val Loss=0.2984
Mean F1 Score: 0.9914, Mean Accuracy: 0.9888, Mean Precision: 0.9901, Mean Recall: 0.9930


C:\Users\udays\AppData\Local\Temp\ipykernel_22228\3925381797.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 100 Validation: 100%|█████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.42it/s, loss=0.0717]


Epoch 100: Train Loss=0.2305, Val Loss=0.3020
Mean F1 Score: 0.9933, Mean Accuracy: 0.9910, Mean Precision: 0.9885, Mean Recall: 0.9982
New best model saved for fold 5 with F1=0.9933
